In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import itertools 
import multiprocessing as mp
import numpy as np
import sys

sys.path.append('../')
from baseModels.SVM.model import SVM

In [2]:
feature_dir = '../featEngg/kernelMethods/data/featvec/'
feature_files = [f.name for f in os.scandir(feature_dir) if f.name.endswith('.csv.gz')]

In [3]:
label_file = '../data/LabelFiles/EnzymeLabelsMultiClass.csv'

In [4]:
def model_evaluate(filename,random_seed):
    df1 = pd.read_csv(feature_dir+filename,header=None)
    df2 = pd.read_csv(label_file,header=None)
    df = df1.merge(df2,on=0)
    enz_names = df[0].values
    X = df.iloc[:,1:-1].values
    y = df.iloc[:,-1].values
    if X.shape[1]<55:
        n_comp = int(0.75*X.shape[1])
    else:
        n_comp=55
    X_train, X_test, y_train, y_test,enz_train,enz_test = train_test_split(X, y,enz_names, test_size=0.25, random_state=random_seed)
    svm = SVM(X_train,X_test,y_train,y_test,verbose=False,optimize=False, pca_comp=n_comp,kern='rbf',regC=30)
    return svm.acc_train,svm.acc_test
 

In [5]:
mp.cpu_count()

24

In [6]:
pool = mp.Pool(mp.cpu_count())

In [7]:
iterable = list(itertools.product(feature_files,range(1000)))

In [8]:
%%time
model_accs = list(pool.starmap(model_evaluate,iterable))

CPU times: user 339 ms, sys: 107 ms, total: 446 ms
Wall time: 5min 8s


In [9]:
mydict = dict()

for m,acc in zip(iterable,model_accs):
    model_name=m[0].replace('.csv.gz','')
    seed = m[1]
    if model_name not in mydict:
        mydict[model_name] = {seed:acc}
    else:
        mydict[model_name][seed] = acc

In [10]:
def get_model_stats(mname):
    train_accs = [v[0] for k,v in mydict[mname].items()]
    test_accs = [v[1] for k,v in mydict[mname].items()]
    return {'train_min':min(train_accs),'train_mean':np.mean(train_accs),
            'train_max':max(train_accs),'train_std':np.std(train_accs),
            'test_min':min(test_accs),'test_mean':np.mean(test_accs),
            'test_max':max(test_accs),'test_std':np.std(test_accs)}

In [11]:
get_model_stats('mismatchKernel')

{'train_min': 0.9069767441860465,
 'train_mean': 0.9503604651162793,
 'train_max': 0.9883720930232558,
 'train_std': 0.013781100821755267,
 'test_min': 0.4827586206896552,
 'test_mean': 0.7621724137931035,
 'test_max': 0.9655172413793104,
 'test_std': 0.07220323245981207}

In [12]:
results_dict= {mname:get_model_stats(mname) for mname in mydict.keys()}
    

In [13]:
results_df = pd.DataFrame(results_dict).T

In [14]:
results_df.to_csv('../data/SimResults/KernelResults.csv',index=True)

In [15]:
results_df.sort_values('train_mean')

,train_min,train_mean,train_max,train_std,test_min,test_mean,test_max,test_std
mismatchKernel,0.906977,0.950360,0.988372,0.013781,0.482759,0.762172,0.965517,0.072203
gappyKernel,0.930233,0.968709,1.000000,0.012070,0.482759,0.758345,0.965517,0.073876
spectrumKernel,0.930233,0.970209,1.000000,0.011733,0.482759,0.756759,0.965517,0.074223
